In [ ]:
import os

import hydra
import rootutils
from hydra import compose, initialize
from omegaconf import OmegaConf

rootutils.setup_root(os.getcwd(), indicator=".project-root", pythonpath=True)
OmegaConf.register_new_resolver("len", len)
OmegaConf.register_new_resolver("eval", eval)

with initialize(version_base="1.3", config_path="../configs"):
    cfg = compose(config_name="train.yaml", return_hydra_config=True)

data = hydra.utils.instantiate(cfg.data)
# model = hydra.utils.instantiate(cfg.model)

In [ ]:
import pickle

import torch

filepath = r"C:\Users\amedee.roy\OneDrive - FRANCE ENERGIES MARINES\SCRIPTS\defile-migration-forecast\logs\train\runs\2025-03-27_11-32-43\Common Buzzard\test_explanation.pickle"
with open(filepath, "rb") as f:
    test_explanation = pickle.load(f)

test_explanation = [
    torch.cat([exp[k] for exp in test_explanation], dim=0) for k in range(len(test_explanation[0]))
]
yr, doy, era5_main, era5_hourly, era5_daily = test_explanation

In [ ]:
from matplotlib import pyplot as plt

plt.hist(yr.squeeze().numpy(), edgecolor="k", alpha=0.5)
plt.title("year")
plt.show()

plt.hist(doy.squeeze().numpy(), edgecolor="k", alpha=0.5)
plt.title("day of year")
plt.show()

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(6, 10), tight_layout=True, sharex=True)

values = torch.mean(era5_main, dim=(0, 2, 3)).numpy()
variables = list(data.era5_main.data_vars)
ax[0].barh(variables, values)
ax[0].set_title("Local hourly metrics")

values = torch.mean(era5_hourly, dim=(0, 2, 3)).numpy()
variables = list(data.era5_hourly.data_vars)
ax[1].barh(variables, values)
ax[1].set_title("Remote hourly metrics")

values = torch.mean(era5_daily, dim=(0, 2, 3)).numpy()
variables = list(data.era5_daily.data_vars)
ax[2].barh(variables, values)
ax[2].set_title("Remote daily metrics")
plt.show()

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(6, 6), tight_layout=True, sharex=True)

values = torch.mean(era5_hourly, dim=(0, 1, 2)).numpy()
variables = list(data.era5_hourly.location.values)
ax[0].barh(variables, values)
ax[0].set_title("Remote hourly locations")

values = torch.mean(era5_daily, dim=(0, 1, 2)).numpy()
variables = list(data.era5_daily.location.values)
ax[1].barh(variables, values)
ax[1].set_title("Remote daily locations")
plt.show()